## In this notebook, we will load and clean our data
 We are working with 10 files, but df_train_all_v1.parquet is the full training data set we can start with



In [ ]:
#add libraries to use

In [ ]:
# Load in df_train_all_v1.parquet



In [ ]:
#Check its shape, types and missing values